In [1]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 0.24.2
Uninstalling scikit-learn-0.24.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.24.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/libgomp-f7e03b3e.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.24.2


In [5]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==0.24.2; python_version <= "3.7", but you have scikit-learn 1.0.2 which is incompatible.


In [6]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [9]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [10]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

In [11]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['filename', 'labels'])

In [12]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [13]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [14]:
#Inserting filename and label values into dataframe
dataframe['filename'] = filepathslist
dataframe['labels'] = labels

In [15]:
dataframe

,filename,labels
0,1.wav,0
1,2.wav,0
2,3.wav,0
3,4.wav,0
4,5.wav,0
5,6.wav,1
6,7.wav,1
7,8.wav,1
8,9.wav,1
9,10.wav,1


In [16]:
#Computing Features - 

psdlist = list()
rmslist = list()
zcrlist = list()

#Discrete Wavelet Transform - 
#Output is a dict of lists - 
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    coeffnamelist = ['camean', 'castdev', 'caskew', 'cakurtosis', 'cdmean', 'cdstdev', 'cdskew', 'cdkurtosis']
    dwtlist = getdwtlist(res, coeffnamelist)
    return dwtlist

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
#Output is a list - 
def computepsd(x):
    freq, psd = signal.welch(x)
    psdlist.append(psd)
    return psdlist

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
#Output is a dict of lists - 
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    mfccdict = getmfcclist(mfccs)
    return mfccdict

#Computing the gammatone frequency cepstral coefficients - 
#Output is a dict of lists - 
def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    dictgfcc = getgfcclist(gfcc, file_name)
    return dictgfcc

#Computing rms, zcr, and general stats - 

#Output is a list - 
def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    rmslist.append(rmsen[0])
    return rmslist

#Output is a list - 
def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    zcrlist.append(zcr[0])
    return zcrlist

#Output is a dict of lists - 
def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    coeffnamelist = ['mean', 'variance', 'skew', 'kurtosis']
    statlist = getstatlist(stats, coeffnamelist)
    return statlist

In [17]:
#Making lists for each feature - 

dictofmfcc = defaultdict(list)
dictofgfccs = defaultdict(list)
dictofdwt = defaultdict(list)
dictofstat = defaultdict(list)

def getmfcclist(mfccs):
  for i in range(13):
    item = mfccs[i]
    dictofmfcc[f"mfcc_{i}"].append(item)
  return dictofmfcc

def getgfcclist(gfccs, filename):
  for i in range(44):
    fn = gfccs['audio'][filename]['feature_names'][i]
    item = gfccs['audio'][filename]['features'][i]
    dictofgfccs[fn].append(item)
  return dictofgfccs

def getdwtlist(res, coefflist):
  for i in range(8):
    fn = coefflist[i]
    item = res[i]
    dictofdwt[fn].append(item)
  return dictofdwt

def getstatlist(res, coefflist):
  for i in range(4):
    fn = coefflist[i]
    item = res[i]
    dictofstat[fn].append(item)
  return dictofstat

In [18]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
    if dwt:
        dwtlist = computeDWTs(x)
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
    if psd:
        psds = computepsd(x)
    if gfcc:
        gfccs = computeGFCC(file_name)
    if rmse:
        rmselist = computeRMSEn(x)
    if zcr:
        zcrslist = computeZCR(x)
    if stats:
        statslist = computestats(x)
    return mfccs, dwtlist, psds, gfccs, rmselist, zcrslist, statslist

In [19]:
#loads audio and extracts required features
def load_file_data(file_names, duration=5, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    for file_name in file_names:
        try:
            #sound_file = folder + '/' + file_name
            sound_file = file_name
            print ("load file ",sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_name)
                y = librosa.util.fix_length(x, input_length)
            dictmfcc, dictdwt, listpsd, dictgfcc, listrms, listzcr, dictstats = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)           
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
    return dictmfcc, dictdwt, listpsd, dictgfcc, listrms, listzcr, dictstats

In [20]:
dictmfcc, dictdwt, listpsd, dictgfcc, listrms, listzcr, dictstats = load_file_data(filepathslist)

load file  1.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 1.wav
load file  2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 2.wav
load file  3.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 3.wav
load file  4.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 4.wav
load file  5.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 5.wav
load file  6.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 6.wav
load file  7.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 7.wav
load file  8.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 8.wav
load file  9.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 9.wav
load file  10.wav

        
 Extracti

In [21]:
dictmfcc

defaultdict(list,
            {'mfcc_0': [-467.0059,
              -547.4695,
              -493.46152,
              -477.09448,
              -501.43863,
              -516.0458,
              -483.41226,
              -493.143,
              -489.08957,
              -465.3647,
              -446.08536,
              -435.32278,
              -499.64505,
              -478.02737,
              -471.603,
              -465.18365,
              -437.15393,
              -478.02737,
              -478.039,
              -473.92462,
              -487.73807,
              -487.0935,
              -458.8938],
             'mfcc_1': [30.030811,
              46.999607,
              45.738007,
              41.272854,
              40.63142,
              55.697857,
              94.81681,
              82.83153,
              96.131874,
              54.882587,
              69.64291,
              83.881805,
              62.77168,
              29.280779,
              56.741844,
     

In [22]:
featuredict = [dictmfcc, dictdwt, dictgfcc, dictstats]
featurelist = [listpsd, listrms, listzcr]
namelist = ['psd', 'rms', 'zcr']

In [23]:
#for i in featuredict:
 # key = i.keys()
  #for j in key:
   # val = i[j]
    #fdf[j] = val

#listnames = ['psd', 'rms', 'zcr']

#j = 0
#for i in featurelist:
 # col = listnames[j]
  #fdf[col] = i
  #j = j+1

In [27]:
def getfeaturedataframe(dictfeat, listfeat, listnames):
  Featuredataframe = pd.DataFrame(columns = ['Wav ID'])
  Featuredataframe['Wav ID'] = filepathslist

  for i in dictfeat:
    key = i.keys()
    for j in key:
      val = i[j]
      Featuredataframe[j] = val
  
  k = 0
  for i in listfeat:
    col = listnames[k]
    Featuredataframe[col] = i
    k = k+1
  
  Featuredataframe['Labels'] = labels
  return Featuredataframe

In [28]:
Featuresdf = getfeaturedataframe(featuredict, featurelist, namelist)

In [29]:
Featuresdf

,Wav ID,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,gfcc_21_std,gfcc_22_std,mean,variance,skew,kurtosis,psd,rms,zcr,Labels
0,1.wav,-467.005890,30.030811,28.432730,25.964861,22.885386,19.493237,16.082209,12.900651,10.124111,...,0.017984,0.014769,-0.000917,0.012629,-0.025461,32.262532,"[0.45874533, 1.8715777, 0.43308386, 0.03925463...",0.054259,0.006085,0
1,2.wav,-547.469482,46.999607,43.693832,38.678108,32.594315,26.166515,20.074808,14.848034,10.796354,...,0.018575,0.015226,-0.000917,0.003868,-0.021340,10.566409,"[0.07342167, 0.5139902, 0.099655464, 0.0055495...",0.038230,0.007311,0
2,3.wav,-493.461517,45.738007,43.258198,39.384975,34.471062,28.942736,23.245995,17.793795,12.922318,...,0.020262,0.016092,-0.000919,0.010944,-0.011584,12.579745,"[0.34817615, 1.5461591, 0.33767405, 0.02169116...",0.067872,0.009874,0
3,4.wav,-477.094482,41.272854,37.960964,33.038132,27.242409,21.343788,15.981769,11.558811,8.211711,...,0.018572,0.014939,-0.000915,0.012200,-0.023764,21.344287,"[0.45441374, 1.817578, 0.45538217, 0.035509463...",0.060943,0.007378,0
4,5.wav,-501.438629,40.631420,38.138077,34.408627,29.948507,25.144924,20.404640,16.023802,12.308167,...,0.021733,0.017771,-0.000907,0.010723,-0.006688,12.174772,"[0.38234735, 1.5605575, 0.34770936, 0.02111675...",0.065149,0.017013,0
5,6.wav,-516.045776,55.697857,51.505856,45.195385,37.594936,29.574570,21.893900,15.111688,9.555058,...,0.016438,0.013263,0.000765,0.002854,-0.090004,26.059404,"[0.11527796, 0.4288319, 0.10983303, 0.01023379...",0.030211,0.022660,1
6,7.wav,-483.412262,94.816811,75.374596,50.192509,26.567419,9.711957,1.201846,-0.620575,1.064039,...,0.017324,0.014199,0.000925,0.003792,-0.113119,17.879098,"[0.113630705, 0.55274874, 0.1274686, 0.0146120...",0.040199,0.014911,1
7,8.wav,-493.143005,82.831528,67.474709,47.141670,27.380001,12.501788,4.271763,1.873725,2.926418,...,0.013492,0.011121,0.000961,0.004659,-0.094476,18.481855,"[0.1841076, 0.70758694, 0.16161688, 0.00363750...",0.040763,0.011841,1
8,9.wav,-489.089569,96.131874,75.565567,49.462837,25.198654,7.879220,-0.771518,-2.355857,-0.207289,...,0.015081,0.012252,0.001103,0.004365,-0.058234,15.431743,"[0.13849796, 0.65075976, 0.1458105, 0.00788094...",0.041749,0.015340,1
9,10.wav,-465.364685,54.882587,46.444679,34.988621,23.336475,13.805823,7.535925,4.392777,3.385290,...,0.018502,0.014812,-0.000916,0.012097,-0.001949,21.833281,"[0.65056837, 1.8061801, 0.47678462, 0.02850866...",0.060470,0.013206,1


In [30]:
Featuresdf.to_csv('featuredata.csv')